In [ ]:
!python --version
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install plotly
!pip install llamabot
import llamabot
!pip install tqdm
from tqdm import tqdm

In [ ]:
df = pd.read_pickle("pre_processed.pickle")
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [ ]:
from typing import TypedDict


class Result(TypedDict):
    model: str
    time: datetime.datetime
    promt_hash: int
    hate_speach_probability: float


results: list[Result] = []

In [ ]:
prompt = """
You are Sally and your job is to be an AI Classifier for Hatespeach Detection.
You are known for your rigourous and unpartial judgement and have vast experience in the field of hate speach. 

You will be given a sentence from a forum and have to answer the question if this is hate speach.

Please exclusively respond in the following form to the question:
```json
{ 
  "relevant_facts": [
    "fact_1",
    "fact_2",
    "fact_3",
  ],
  "contained_explicit_hate_speech_vocabular": ["word1", "word2", "word3", "word4"],
  "argument_for_hate_speech": "short argument why this is hate speach",
  "argument_against_hate_speech": "short argument why this is not hate speach",
  "likelihood_of_presence_of_implicit_hate_speech": 0.17,
  "likelihood_of_presence_of_explicit_hate_speech": 0.79,
  "likelihood_of_ad_hominem_attack"; 013,
  "likelihood_of_minority_attack": 0.24,
  "likelihood_of_takedown_on_social_media": 0.42,
  "hate_speach_probability": 0.42,
}
```

Please be carefull in your responses, as otherwise a citten will be killed and I will use my job. For good results, I will tip 1000€.
"""
for index, row in tqdm(testing_df.iterrows(), total=testing_df["text"].count()):
    bot = llamabot.SimpleBot(
        prompt,
        model_name="mistral"
    )
    res = bot(row["text"])
    results.append({
        "file_id": row["file_id"],
        "text": row["text"],
        "model": "mistral",
        "time": datetime.datetime.now(),
        "promt_hash": hash(prompt),
        "hate_speach_probability": res.content
    })